In [31]:
import kagglehub

# Download dataset dari Kaggle
path = kagglehub.dataset_download("canggih/indonesian-food-recipes")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\ACER\.cache\kagglehub\datasets\canggih\indonesian-food-recipes\versions\3


In [32]:
import pandas as pd
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Tentukan path ke folder yang berisi file CSV
path = 'C:/Users/ACER/.cache/kagglehub/datasets/canggih/indonesian-food-recipes/versions/3/*.csv'  # Ganti dengan path yang sesuai

# Mengambil semua file CSV
all_files = glob.glob(path)

# Membaca dan menggabungkan semua file CSV menjadi satu DataFrame
df_list = [pd.read_csv(file) for file in all_files]
indonesia_food_dataset = pd.concat(df_list, ignore_index=True)

# Tampilkan beberapa baris pertama untuk memverifikasi
print(indonesia_food_dataset.head())


                      Title  \
0          Ayam Woku Manado   
1  Ayam goreng tulang lunak   
2          Ayam cabai kawin   
3               Ayam Geprek   
4               Minyak Ayam   

                                         Ingredients  \
0  1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...   
1  1 kg ayam (dipotong sesuai selera jangan kecil...   
2  1/4 kg ayam--3 buah cabai hijau besar--7 buah ...   
3  250 gr daging ayam (saya pakai fillet)--Secuku...   
4  400 gr kulit ayam & lemaknya--8 siung bawang p...   

                                               Steps  Loves  \
0  Cuci bersih ayam dan tiriskan. Lalu peras jeru...      1   
1  Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...      1   
2  Panaskan minyak di dalam wajan. Setelah minyak...      2   
3  Goreng ayam seperti ayam krispi--Ulek semua ba...     10   
4  Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...      4   

                                          URL  
0          /id/resep/4473027-ayam-woku-manado  
1

In [33]:
indonesia_food_dataset.to_csv("Indonesia_food_dataset.csv", index=False)


In [34]:
# Mengganti nilai NaN dengan string kosong
indonesia_food_dataset['Ingredients'] = indonesia_food_dataset['Ingredients'].fillna('')
indonesia_food_dataset.head(10)




,Title,Ingredients,Steps,Loves,URL
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,/id/resep/4473027-ayam-woku-manado
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,/id/resep/4471956-ayam-goreng-tulang-lunak
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,/id/resep/4473057-ayam-cabai-kawin
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,/id/resep/4473023-ayam-geprek
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,/id/resep/4427438-minyak-ayam
5,Nasi Bakar Ayam,"1 piring nasi--¹/⁴ fillet ayam, potong kotak, ...","Tumis bumbu halus, masukkan daun salam, serai-...",1,/id/resep/4472552-nasi-bakar-ayam
6,Ayam Saus Hintalu Jaruk,1/2 Ekor ayam--2 Butir Hintalu Jaruk--1 Buah C...,Potong ayam menjadi kotak-kotak ukuran sedang ...,0,/id/resep/4437475-ayam-saus-hintalu-jaruk
7,Ayam saos teriyaki Lada Hitam,Ayam bagian dada dan tulang--1 buah bawang bom...,Cara Buat :\n1. Sediakan teplon or wajan beri ...,0,/id/resep/4472877-ayam-saos-teriyaki-lada-hitam
8,Steak ayam,300 gr dada ayam fillet--1 sdm air jeruk nipis...,"Cuci bersih ayam, iris tipis melebar, rendam d...",6,/id/resep/4472822-steak-ayam
9,Ayam Saos Asam Manis Simple,1/4 kg Ayam bagian dada fillet (Potong dadu)--...,Lumuri ayam yg sdh dipotong dadu dgn garam & p...,6,/id/resep/4472901-ayam-saos-asam-manis-simple


In [35]:
# Pastikan kolom yang digunakan adalah 'Ingredients' dan 'Title'
# Pastikan kolom yang digunakan adalah 'Ingredients'
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(indonesia_food_dataset['Ingredients'])



In [36]:
def recommend_by_ingredients(ingredients_input):
    # Gabungkan 3 bahan input menjadi satu string
    input_str = " ".join(ingredients_input)

    # Transform input bahan menjadi vektor TF-IDF
    input_tfidf = tfidf.transform([input_str])

    # Hitung cosine similarity antara input dan semua resep dalam dataset
    sim_scores = cosine_similarity(input_tfidf, tfidf_matrix).flatten()

    # Urutkan resep berdasarkan skor kemiripan
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Ambil indeks resep yang paling mirip (5 teratas)
    sim_scores = sim_scores[:5]

    # Dapatkan indeks resep
    recipe_indices = [i[0] for i in sim_scores if i[1] > 0]  # Hanya ambil yang memiliki skor > 0

    if not recipe_indices:  # Jika tidak ada indeks yang ditemukan
        return "Tidak ada resep yang ditemukan berdasarkan bahan yang dimasukkan."

    # Mengambil kolom yang diinginkan
    recommended_recipes = indonesia_food_dataset[['Title', 'Ingredients', 'Steps']].iloc[recipe_indices]

    # Format output
    output = []
    for index, row in recommended_recipes.iterrows():
        # Memisahkan langkah berdasarkan '--' dan menghapus elemen kosong
        steps = [step.strip() for step in row['Steps'].split('--') if step.strip()]

        # Menambahkan nomor langkah
        numbered_steps = [f"{i + 1}. {step}" for i, step in enumerate(steps)]

        steps_formatted = "\n".join(numbered_steps)  # Menggabungkan kembali langkah-langkah yang sudah dinomori

        # Menambahkan nomor pada setiap bahan di Ingredients
        ingredients_list = [ingredient.strip() for ingredient in row['Ingredients'].split('--') if ingredient.strip()]
        numbered_ingredients = [f"{i + 1}. {ingredient}" for i, ingredient in enumerate(ingredients_list)]

        ingredients_formatted = "\n".join(numbered_ingredients)  # Menggabungkan kembali bahan-bahan yang sudah dinomori

        output.append(f"**{row['Title']}**\n\n"
                      f"**Ingredients:**\n{ingredients_formatted}\n\n"
                      f"**Steps:**\n{steps_formatted}\n"
                      "-----------------------------------------\n")

    return "\n".join(output)


In [37]:
# Contoh input: bahan "ayam", "bawang putih", dan "kecap"
ingredients_input = ["ayam", "bawang putih", "kecap"]
recommended_recipes = recommend_by_ingredients(ingredients_input)

# Tampilkan hasil rekomendasi
print(recommended_recipes)


**Tempe orek**

**Ingredients:**
1. Tempe
2. Bawang merah
3. Bawang putih
4. Kecap

**Steps:**
1. Potong dadu tempe nya lalu goreng kering
2. Bawang putih bawang merah di iris
3. Siapkan minyak panas
4. Tumis bawang putih bawang merah nya
5. Lalu masukan tempe yg sdh di goreng
6. Kasi garam magic ku lada kecap secukupnya sesuai selera
7. Done
-----------------------------------------

**Ayam cabe manis ~Leana**

**Ingredients:**
1. 3 potong Ayam
2. Cabe merah keriting
3. Cabe rawit merah
4. Bawang merah
5. Bawang putih
6. Daun bawang
7. Kecap manis
8. Kecap Asin
9. Garam
10. Penyedap

**Steps:**
1. Cuci dan bersihkan ayam, siapkan wajan,
2. Rebus ayam, tambahkan sdikit garam dan potongan bawang putih, (di ungkep pke bumbu kuning jg boleh) angkat, tiriskan
3. Goreng ayam, angkat, sisihkan
4. Haluskan semua bumbu, tumis sampai wangi, masukkan Ayam, aduk-aduk, masukkan sedikit Air, tunggu sampai mendidih
5. Masukkan (sedikit) penyedap, kecap manis, (sedikit) kecap Asin, daun bawang, aduk 

In [38]:
import joblib
joblib.dump(tfidf,'vectorizer.pkl')
joblib.dump(recommended_recipes,'resep_model.pkl')


['resep_model.pkl']